In [1]:
from fam.llm.fast_inference import TTS

import warnings

# Filter out user warnings
warnings.filterwarnings("ignore")

/root/kotoba-speech-release/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using dtype=bfloat16


/root/kotoba-speech-release/myenv/lib/python3.10/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [2]:
model = TTS(model_name='metavoiceio/metavoice-1B-v0.1')

using dtype=bfloat16


Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 24082.13it/s]


number of parameters: 14.07M
2024-04-25 02:12:28 | INFO     | DF | Running on torch 2.1.0+cu121
2024-04-25 02:12:28 | INFO     | DF | Running on host 627a2bd7686c
2024-04-25 02:12:28 | INFO     | DF | Git commit: 866b660, branch: release
2024-04-25 02:12:28 | INFO     | DF | Loading model settings of DeepFilterNet3
2024-04-25 02:12:28 | INFO     | DF | Using DeepFilterNet3 model at /root/.cache/DeepFilterNet/DeepFilterNet3
2024-04-25 02:12:28 | INFO     | DF | Initializing model `deepfilternet3`
2024-04-25 02:12:28 | INFO     | DF | Found checkpoint /root/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2024-04-25 02:12:28 | INFO     | DF | Running on device cuda:0
2024-04-25 02:12:28 | INFO     | DF | Model loaded
/root/.cache/huggingface/hub/models--metavoiceio--metavoice-1B-v0.1/snapshots/d7237c5cb970da187623738f2957b338f3b0889a/first_stage.pt
Using device=cuda
Loading model ...
using dtype=bfloat16
Time to load model: 3.80 seconds
Compiling...Can t

100%|██████████| 199/199 [00:28<00:00,  7.09it/s]

Compilation time: 65.15 seconds


## **Testing the new synthesizer** ##

In [15]:
model.synthesiseSequential(text="The university of waterloo is a publicly funded research university located in ontario canada.", spk_ref_path="/root/kotoba-speech-release/fam/llm/waterloov2.mp3")

 63%|██████▎   | 1262/1994 [00:17<00:09, 73.82it/s]


Time for 1st stage LLM inference: 17.12 sec total, 73.78 tokens/sec
Bandwidth achieved: 184.22 GB/s
Memory used: 20.10 GB

Truncating to min length of 630 tokens from 632 max.
[632, 630]


non-causal batching: 100%|██████████| 1/1 [00:00<00:00, 175.64it/s]


Text: The university of waterloo is a publicly funded research university located in ontario canada.

Saved audio to /root/kotoba-speech-release/fam/outputs/synth_The_university_of_waterlo_5fac4efc-0d47-4b3a-83c5-5fef86c8602d.wav

Saved audio to /root/kotoba-speech-release/fam/outputs/synth_The_university_of_waterlo_5fac4efc-0d47-4b3a-83c5-5fef86c8602d.wav

Total time to synth (s): 21.697964191436768
Real-time factor: 2.58


'/root/kotoba-speech-release/fam/outputs/synth_The_university_of_waterlo_5fac4efc-0d47-4b3a-83c5-5fef86c8602d.wav'

In [4]:
model.synthesiseSequential(text="I have a black cat and waterloo is cool.", spk_ref_path="/root/kotoba-speech-release/fam/llm/black_cat.mp3")

 23%|██▎       | 466/2027 [00:06<00:21, 74.20it/s]


Time for 1st stage LLM inference: 6.30 sec total, 74.09 tokens/sec
Bandwidth achieved: 184.99 GB/s
Memory used: 15.49 GB

Truncating to min length of 232 tokens from 234 max.
[234, 232]


non-causal batching: 100%|██████████| 1/1 [00:00<00:00, 164.99it/s]


Text: I have a black cat and waterloo is cool.

Saved audio to /root/kotoba-speech-release/fam/outputs/synth_I_have_a_black_cat_and_wa_a893066c-83fe-4e0b-ade2-062bd63bd669.wav

Saved audio to /root/kotoba-speech-release/fam/outputs/synth_I_have_a_black_cat_and_wa_a893066c-83fe-4e0b-ade2-062bd63bd669.wav

Total time to synth (s): 8.37227463722229
Real-time factor: 2.71


'/root/kotoba-speech-release/fam/outputs/synth_I_have_a_black_cat_and_wa_a893066c-83fe-4e0b-ade2-062bd63bd669.wav'

**Testing audio tokenizer**

In [5]:
from fam.llm.decoders import EncodecDecoder
from fam.llm.inference import TiltedEncodec
import torch
from pathlib import Path
from huggingface_hub import snapshot_download
from fam.quantiser.text.tokenise import TrainedBPETokeniser

data_adapter_second_stage = TiltedEncodec(end_of_audio_token=1024)
_other_model_dir = snapshot_download(repo_id="metavoiceio/metavoice-1B-v0.1")
checkpoint_path = Path(f"{_other_model_dir}/first_stage.pt")

checkpoint = torch.load(str(checkpoint_path), mmap=True, weights_only=False)
tokenizer_info = checkpoint.get("meta", {}).get("tokenizer", {})
tokenizer = TrainedBPETokeniser(**tokenizer_info)

enc = EncodecDecoder(data_adapter_fn=data_adapter_second_stage.decode, output_dir="/root/kotoba-speech-release/fam/outputs", tokeniser_decode_fn=tokenizer)

toks = enc.get_tokens(audio_path="/root/kotoba-speech-release/fam/llm/waterloostuffs.mp3")

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 71089.90it/s]


In [6]:
print(tokenizer.encode("Hey there goose man"))

[2453, 2170, 2517, 2347, 2404, 2365, 2482, 2561]


In [8]:
len(toks)

8

In [9]:
len(toks[0])

548